In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            35G        901M         32G        1.1M        2.1G         33G
Swap:            0B          0B          0B


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [6]:
!wget -c https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2021-08-30 00:04:12--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2021-08-30 00:04:12--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Reusing existing connection to rajpurkar.github.io:443.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [115]:
!wget -c https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/ -O evaluate-v2.0.py

--2021-08-30 00:53:05--  https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/
Resolving worksheets.codalab.org (worksheets.codalab.org)... 13.68.212.115
Connecting to worksheets.codalab.org (worksheets.codalab.org)|13.68.212.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: codalab_session=""; expires=Thu, 01 Jan 1970 00:00:00 GMT; Max-Age=-1; Path=/ at position 70.
Length: unspecified [text/x-python]
Saving to: ‘evaluate-v2.0.py’

evaluate-v2.0.py        [ <=>                ]  10.30K  --.-KB/s    in 0.001s  

2021-08-30 00:53:05 (7.31 MB/s) - ‘evaluate-v2.0.py’ saved [10547]



In [7]:
!git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [19]:
!pip install transformers[sentencepiece]

In [9]:
!ls

 cache_dir	    dev-v2.0.json   train-v2.0.json
'Colab Notebooks'   index.html	    transformers


In [10]:
%cd transformers/examples/legacy/question-answering/

/content/drive/MyDrive/transformers/examples/legacy/question-answering


In [11]:
!ls ../../../..

 cache_dir	    dev-v2.0.json   train-v2.0.json
'Colab Notebooks'   index.html	    transformers


In [12]:
!ls .

cached_train_distilbert-base-uncased_384  run_squad.py
runs					  run_squad_trainer.py


# Fine-Tune albert model on Squad

In [ ]:
!python run_squad.py \
  --model_type albert \
  --model_name_or_path albert-xlarge-v2 \
  --tokenizer_name albert-xlarge-v2 \
  --do_train \
  --do_eval \
  --do_lower_case \
  --train_file ../../../../train-v2.0.json \
  --predict_file ../../../../dev-v2.0.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --save_steps 300 \
  --cache_dir ../../../../cache_dir/ \
  --output_dir ../../../../checkpoint/

# Use pretrained ones

# List of good models to finetune or download as pretrained

In [16]:
# List of Greate models in QA task

# ALBERT 
# BIOBERT
# microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext (Pytorch)
# microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract (Pytorch)
# dmis-lab/biobert-v1.1 (Pytorch)
# dmis-lab/biobert-base-cased-v1.1 (Pytorch)
# dmis-lab/biobert-large-cased-v1.1-squad (Pytorch)
# ktrapeznikov/biobert_v1.1_pubmed_squad_v2 (Pytorch)
# franklu/pubmed_bert_squadv2 (Pytorch)
# ktrapeznikov/albert-xlarge-v2-squad-v2 (Pytorch)
# ktrapeznikov/scibert_scivocab_uncased_squad_v2 (Pytorch)

In [39]:
import torch
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TFAutoModelForQuestionAnswering

# Albert model

In [93]:
albert_tokenizer = AutoTokenizer.from_pretrained("ahotrod/albert_xxlargev1_squad2_512")
albert_model = TFAutoModelForQuestionAnswering.from_pretrained("ahotrod/albert_xxlargev1_squad2_512")

All model checkpoint layers were used when initializing TFAlbertForQuestionAnswering.

All the layers of TFAlbertForQuestionAnswering were initialized from the model checkpoint at ahotrod/albert_xxlargev1_squad2_512.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertForQuestionAnswering for predictions without further training.


In [94]:
context = """Use acetaminophen exactly as directed on the label, or as prescribed by your doctor. Do not use in larger or smaller amounts or for longer than recommended.
Do not take more of this medication than is recommended. An overdose of acetaminophen can damage your liver or cause death.
Adults and teenagers who weigh at least 110 pounds (50 kilograms): Do not take more than 1000 milligrams (mg) at one time. Do not take more than 4000 mg in 24 hours.
Children younger than 12 years old: Do not take more than 5 doses of acetaminophen in 24 hours. Use only the number of milligrams per dose that is recommended for the child's weight and age. Use exactly as directed on the label.
Avoid also using other medicines that contain acetaminophen, or you could have a fatal overdose.
If you are treating a child, use a pediatric form of acetaminophen. Use only the special dose-measuring dropper or oral syringe that comes with the specific pediatric form you are using. Carefully follow the dosing directions on the medicine label.
Measure liquid medicinewith the dosing syringe provided, or with a special dose-measuring spoon or medicine cup. If you do not have a dose-measuring device, ask your pharmacist for one.
Acetaminophen made for infants is available in two different dose concentrations, and each concentration comes with its own medicine dropper or oral syringe. These dosing devices are not equal between the different concentrations. Using the wrong device may cause you to give your child an overdose of acetaminophen. Never mix and match dosing devices between infant formulations of acetaminophen.
You may need to shake the liquid before each use. Follow the directions on the medicine label."""

question = "Can you give me dosage information of Acetaminophen for children?"

In [96]:
inputs = albert_tokenizer.encode_plus(
    question, 
    context, 
    add_special_tokens=True,
    truncation= True, 
    max_length = 512,
    return_tensors="tf")

input_ids = inputs["input_ids"].numpy()[0]

answer_start_scores, answer_end_scores = albert_model(
    inputs, return_dict=False)

answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
answer_end = (tf.argmax(answer_end_scores, axis=1) + 1).numpy()[0]

answer = albert_tokenizer.convert_tokens_to_string(
    albert_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

answer

'do not take more than 5 doses of acetaminophen in 24 hours'

# Albert large context - Truncated

In [97]:
question = "Can you give me dosage information of Acetaminophen for adults?"

In [98]:
context = """Use acetaminophen exactly as directed on the label, or as prescribed by your doctor. Do not use in larger or smaller amounts or for longer than recommended.
Do not take more of this medication than is recommended. An overdose of acetaminophen can damage your liver or cause death.
Adults and teenagers who weigh at least 110 pounds (50 kilograms): Do not take more than 1000 milligrams (mg) at one time. Do not take more than 4000 mg in 24 hours.
Children younger than 12 years old: Do not take more than 5 doses of acetaminophen in 24 hours. Use only the number of milligrams per dose that is recommended for the child's weight and age. Use exactly as directed on the label.
Avoid also using other medicines that contain acetaminophen, or you could have a fatal overdose.
If you are treating a child, use a pediatric form of acetaminophen. Use only the special dose-measuring dropper or oral syringe that comes with the specific pediatric form you are using. Carefully follow the dosing directions on the medicine label.
Measure liquid medicinewith the dosing syringe provided, or with a special dose-measuring spoon or medicine cup. If you do not have a dose-measuring device, ask your pharmacist for one.
Acetaminophen made for infants is available in two different dose concentrations, and each concentration comes with its own medicine dropper or oral syringe. These dosing devices are not equal between the different concentrations. Using the wrong device may cause you to give your child an overdose of acetaminophen. Never mix and match dosing devices between infant formulations of acetaminophen.
You may need to shake the liquid before each use. Follow the directions on the medicine label.
The chewable tablet must be chewed thoroughly before you swallow it.
Make sure your hands are dry when handling the acetaminophen disintegrating tablet. Place the tablet on your tongue. It will begin to dissolve right away. Do not swallow the tablet whole. Allow it to dissolve in your mouth without chewing.
To use the acetaminophen effervescent granules, dissolve one packet of the granules in at least 4 ounces of water. Stir this mixture and drink all of it right away. To make sure you get the entire dose, add a little more water to the same glass, swirl gently and drink right away.
The oral powder should be placed directly on the tongue and swallowed.
Stop taking acetaminophen and call your doctor if:
you still have a sore throat after 2 days of use;
you still have a fever after 3 days of use;
you still have pain after 7 days of use (or 5 days if treating a child);
you have a skin rash, ongoing headache, nausea, vomiting, or any redness or swelling; or
if your symptoms get worse, or if you have any new symptoms.
This medication can cause unusual results with certain lab tests for glucose (sugar) in the urine. Tell any doctor who treats you that you are using acetaminophen.
Store at room temperature away from heat and moisture.
Detailed Acetaminophen dosage information
What happens if I miss a dose?
Since acetaminophen is taken as needed, you may not be on a dosing schedule. If you are taking the medication regularly, take the missed dose as soon as you remember. Skip the missed dose if it is almost time for your next scheduled dose. Do not take extra medicine to make up the missed dose.
What happens if I overdose?
Seek emergency medical attention or call the Poison Help line at 1-800-222-1222. An overdose of acetaminophen can be fatal.
The first signs of an acetaminophen overdose include loss of appetite, nausea, vomiting, stomach pain, sweating, and confusion or weakness. Later symptoms may include pain in your upper stomach, dark urine, and yellowing of your skin or the whites of your eyes."""

In [99]:
inputs = albert_tokenizer.encode_plus(
    question, 
    context, 
    add_special_tokens=True,
    truncation= True, 
    max_length = 512,
    return_tensors="tf")

input_ids = inputs["input_ids"].numpy()[0]

answer_start_scores, answer_end_scores = albert_model(
    inputs, return_dict=False)

answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
answer_end = (tf.argmax(answer_end_scores, axis=1) + 1).numpy()[0]

answer = albert_tokenizer.convert_tokens_to_string(
    albert_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

answer

'do not take more than 1000 milligrams (mg) at one time. do not take more than 4000 mg in 24 hours'

# Bert

In [100]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
bert_model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [101]:
inputs = bert_tokenizer.encode_plus(
                      question, context, 
                      add_special_tokens=True,     
                      truncation= True, 
                      max_length = 512,
                      return_tensors="tf")

input_ids = inputs["input_ids"].numpy()[0]

answer_start_scores, answer_end_scores = bert_model(
    inputs, return_dict=False)

answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
answer_end = (tf.argmax(answer_end_scores, axis=1) + 1).numpy()[0]

answer = bert_tokenizer.convert_tokens_to_string(
    bert_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

answer

'[SEP]'

# PubMedBert

In [90]:
pubmed_tokenizer = AutoTokenizer.from_pretrained("franklu/pubmed_bert_squadv2")
pubmed_model = AutoModelForQuestionAnswering.from_pretrained("franklu/pubmed_bert_squadv2")

In [91]:
inputs = pubmed_tokenizer.encode_plus(question, 
                              context,                       
                              truncation= True, 
                              max_length = 512,
                              add_special_tokens=True, 
                              return_tensors="pt")

input_ids = inputs["input_ids"].tolist()[0]

text_tokens = pubmed_tokenizer.convert_ids_to_tokens(input_ids)
answer_start_scores, answer_end_scores = pubmed_model(**inputs).values()

answer_start = torch.argmax(
    answer_start_scores
)  
answer_end = torch.argmax(answer_end_scores) + 1 

answer = pubmed_tokenizer.convert_tokens_to_string(pubmed_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))


answer

"do not take more than 1000 milligrams ( mg ) at one time. don't take more than 4000 mg in 24 hours. children younger than 12 years old : don't take more than 5 doses of acetaminophen in 24 hours. use only the number of milligrams per dose that is recommended for the child's weight and age. use exactly as directed on the label"

# Biobert

In [102]:
biobert_tokenizer = AutoTokenizer.from_pretrained("ktrapeznikov/biobert_v1.1_pubmed_squad_v2")
biobert_model = AutoModelForQuestionAnswering.from_pretrained("ktrapeznikov/biobert_v1.1_pubmed_squad_v2")

In [103]:
inputs = biobert_tokenizer.encode_plus(question, 
                              context,                       
                              truncation= True, 
                              max_length = 512,
                              add_special_tokens=True, 
                              return_tensors="pt")

input_ids = inputs["input_ids"].tolist()[0]

text_tokens = biobert_tokenizer.convert_ids_to_tokens(input_ids)
answer_start_scores, answer_end_scores = biobert_model(**inputs).values()

answer_start = torch.argmax(
    answer_start_scores
)  
answer_end = torch.argmax(answer_end_scores) + 1 

answer = biobert_tokenizer.convert_tokens_to_string(biobert_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))


answer

'Do not take more than 1000 milligrams ( mg ) at one time'

# Scibert

In [106]:
scibert_tokenizer = AutoTokenizer.from_pretrained("ktrapeznikov/scibert_scivocab_uncased_squad_v2")
scibert_model = AutoModelForQuestionAnswering.from_pretrained("ktrapeznikov/scibert_scivocab_uncased_squad_v2")

In [107]:
inputs = scibert_tokenizer.encode_plus(question, 
                              context,                       
                              truncation= True, 
                              max_length = 512,
                              add_special_tokens=True, 
                              return_tensors="pt")

input_ids = inputs["input_ids"].tolist()[0]

text_tokens = scibert_tokenizer.convert_ids_to_tokens(input_ids)
answer_start_scores, answer_end_scores = scibert_model(**inputs).values()

answer_start = torch.argmax(
    answer_start_scores
)  
answer_end = torch.argmax(answer_end_scores) + 1 

answer = scibert_tokenizer.convert_tokens_to_string(scibert_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))


answer

"adults and teenagers who weigh at least 110 pounds ( 50 kilograms ) : don't take more than 1000 milligrams ( mg ) at one time"

# Longformer

In [28]:
long_tokenizer = AutoTokenizer.from_pretrained("mrm8488/longformer-base-4096-finetuned-squadv2")
long_model = TFAutoModelForQuestionAnswering.from_pretrained("mrm8488/longformer-base-4096-finetuned-squadv2")

All model checkpoint layers were used when initializing TFLongformerForQuestionAnswering.

All the layers of TFLongformerForQuestionAnswering were initialized from the model checkpoint at mrm8488/longformer-base-4096-finetuned-squadv2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerForQuestionAnswering for predictions without further training.


In [29]:
inputs = long_tokenizer.encode_plus(
    question, context, add_special_tokens=True, return_tensors="tf")

input_ids = inputs["input_ids"].numpy()[0]

answer_start_scores, answer_end_scores = long_model(
    inputs, return_dict=False)

answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
answer_end = (tf.argmax(answer_end_scores, axis=1) + 1).numpy()[0]

answer = long_tokenizer.convert_tokens_to_string(
    long_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

In [30]:
answer

'<s>'